# Online Shopping DBMS - Omer Seyfeddin Koc

In [ ]:
import tkinter as tk
from tkinter import ttk
import pymysql
from tkinter import messagebox
from PIL import Image, ImageTk
import requests
from io import BytesIO


# Function to create a database connection
def create_connection():
    return pymysql.connect(host='localhost', user='root', password='', db='OnlineShopping')

# Function to handle the login process
def login(email, password):
    conn = create_connection()  # Create a connection to the database
    cursor = conn.cursor()
    # Execute SQL query to find the customer with the given email and password
    cursor.execute("SELECT * FROM Customer WHERE Email = %s AND Password = %s", (email, password))
    customer = cursor.fetchone()  # Fetch the first row of the result
    conn.close()
    if customer:
        login_window.destroy()  # Close login window if login is successful
        open_main_app(customer)  # Open the main application window
    else:
        messagebox.showerror("Error", "Invalid credentials")  # Show error if credentials are invalid

# Function to display customer information on the customer tab
def display_customer_info(customer_tab, customer):
    # Nested function to create fields for updating customer information
    def create_update_field(label, current_value, customer_id, column_name):
        def update_value():
            new_value = update_entry.get()  # Get the new value from the entry widget
            update_customer_info(customer_id, column_name, new_value)  # Update the customer info in the database
        
        update_frame = tk.Toplevel()  # Create a new top-level window for updates
        tk.Label(update_frame, text=f"New Updated Value:").pack()
        update_entry = tk.Entry(update_frame)
        update_entry.insert(0, current_value)  # Prefill the entry with the current value
        update_entry.pack()
        tk.Button(update_frame, text="Update", command=update_value).pack()  # Button to trigger the update

    # Dictionary mapping label text to database column names and values
    info_labels = {
        "First Name": ("FirstName", customer[1]),
        "Last Name": ("LastName", customer[2]),
        "Email": ("Email", customer[3]),
        "Phone": ("Phone", customer[4]),
        "Club Points": ("ClubPoints", customer[6]),
        "Password": ("Password", customer[7])
    }

    customer_id = customer[0]  # Customer ID

    # Creating labels and buttons for each customer info field
    for label, (column_name, value) in info_labels.items():
        frame = tk.Frame(customer_tab)
        tk.Label(frame, text=f"{label}:", font=("Helvetica", 16, "bold")).pack(side='left')
        tk.Label(frame, text=value, font=("Helvetica", 16)).pack(side='left')
        if label != "Club Points":  # Exclude 'Club Points' from being updated
            update_button = tk.Button(frame, text="Update", command=lambda col=column_name, val=value: create_update_field(label, val, customer_id, col))
            update_button.pack(side='left')
        frame.pack(anchor='w')
    
    # Frame for displaying profile photo
    photo_frame = tk.Frame(customer_tab)
    photo_frame.pack(side='right', anchor='n', padx=10, pady=10)
    
    # URL for the profile image
    image_url = "https://iili.io/JTOdQIV.png"

    try:
        response = requests.get(image_url)
        if response.status_code == 200:
            img_data = BytesIO(response.content)
            img = Image.open(img_data)
            img = img.resize((300, 300), Image.ANTIALIAS)
            photo = ImageTk.PhotoImage(img)
            # Create a label to display the image and pack it to the right
            profile_label = tk.Label(customer_tab, image=photo)
            profile_label.image = photo  # Keep a reference
            profile_label.pack(side='right', padx=10, pady=10)
        else:
            print("Error: ", response.status_code)

    except Exception as e:
        print("Profile image could not be loaded.")

# Function to update customer information in the database
def update_customer_info(customer_id, column_name, new_value):
    conn = create_connection()  # Establish a database connection
    cursor = conn.cursor()
    query = f"UPDATE Customer SET {column_name} = %s WHERE CustomerID = %s"  # SQL query for updating customer info
    cursor.execute(query, (new_value, customer_id))  # Execute the query with new value and customer ID
    conn.commit()  # Commit the changes to the database
    conn.close()  # Close the database connection
    messagebox.showinfo("Success", f"{column_name} updated successfully")  # Show success message

# Function to display and update a customer's address information
def display_address_info(address_tab, customer_id):
    # Inner function to create a widget for updating address fields
    def create_address_update_field(label, current_value, address_id, column_name):
        def update_address_value():
            new_value = update_entry.get()
            update_address_info(address_id, column_name, new_value)  # Call function to update address info in the database

        update_frame = tk.Toplevel()  # Creates a pop-up window for updating
        tk.Label(update_frame, text=f"New Updated Value:").pack()
        update_entry = tk.Entry(update_frame)
        update_entry.insert(0, current_value)
        update_entry.pack()
        tk.Button(update_frame, text="Update", command=update_address_value).pack()

    # Retrieve the customer's address from the database
    conn = create_connection()
    cursor = conn.cursor()
    cursor.execute("SELECT AddressID, Number, Street, City, State, Zipcode FROM Address WHERE AddressID = (SELECT AddressID FROM Customer WHERE CustomerID = %s)", (customer_id,))
    address = cursor.fetchone()
    conn.close()

    # If address is found, display and provide option to update each field
    if address:
        address_id = address[0]
        info_labels = {
            "Number": ("Number", address[1]),
            "Street": ("Street", address[2]),
            "City": ("City", address[3]),
            "State": ("State", address[4]),
            "Zipcode": ("Zipcode", address[5])
        }
        # Create labels and buttons for address information
        for label, (column_name, value) in info_labels.items():
            frame = tk.Frame(address_tab)
            tk.Label(frame, text=f"{label}:", font=("Helvetica", 16, "bold")).pack(side='left')
            tk.Label(frame, text=value, font=("Helvetica", 16)).pack(side='left')
            update_button = tk.Button(frame, text="Update", command=lambda col=column_name, val=value: create_address_update_field(label, val, address_id, col))
            update_button.pack(side='left')
            frame.pack(anchor='w')

# Function to update address information in the database
def update_address_info(address_id, column_name, new_value):
    conn = create_connection()
    cursor = conn.cursor()
    query = f"UPDATE Address SET {column_name} = %s WHERE AddressID = %s"
    cursor.execute(query, (new_value, address_id))
    conn.commit()
    conn.close()
    messagebox.showinfo("Success", f"{column_name} updated successfully")

# Function to create a new shopping session for a customer
def create_shopping_session(customer_id):
    conn = create_connection()
    cursor = conn.cursor()
    cursor.execute("INSERT INTO ShoppingCart (CustomerID) VALUES (%s)", (customer_id,))
    conn.commit()
    session_id = cursor.lastrowid  # Retrieves the ID of the newly created session
    conn.close()
    return session_id

# Tab for handling shopping sessions
def shopping_session_tab(tab_control, customer):
    shopping_session = ttk.Frame(tab_control)
    tab_control.add(shopping_session, text="Shopping Session")
    # Button to start a new shopping session
    tk.Button(shopping_session, text="Start New Shopping Session", command=lambda: create_shopping_session(customer[0])).pack()

# Global variable to keep track of the total amount in a shopping session
global total_amount
total_amount = 0

# Function to calculate the total amount of items in a shopping session
def calculate_total_amount(session_id):
    conn = create_connection()
    cursor = conn.cursor()
    query = """
    SELECT SUM(p.Price * cp.Quantity) AS Total
    FROM CartProduct cp
    JOIN Product p ON cp.ProductID = p.ProductID
    WHERE cp.ShoppingSessionID = %s
    """
    cursor.execute(query, (session_id,))
    result = cursor.fetchone()
    conn.close()
    return result[0] if result[0] is not None else 0

# Function to update the label showing the total amount in the shopping session
def update_total_amount_label(label, session_id):
    global total_amount
    total_amount = calculate_total_amount(session_id)
    label.config(text=f"Total Amount: {total_amount}")

def display_products(products_tab, session_id):
    # Create a canvas and a scrollbar
    canvas = tk.Canvas(products_tab)
    scrollbar = tk.Scrollbar(products_tab, orient="vertical", command=canvas.yview)
    scrollable_frame = ttk.Frame(canvas)

    # Configure the canvas to use the scrollbar
    canvas.configure(yscrollcommand=scrollbar.set)
    canvas.bind('<Configure>', lambda e: canvas.configure(scrollregion=canvas.bbox("all")))
    
    # Create a window into the canvas that houses the scrollable_frame
    canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")

    # Pack the scrollbar to the right side and fill in the Y direction
    scrollbar.pack(side="right", fill="y")
    # Pack the canvas before the scrollbar fills in the remaining space
    canvas.pack(side="left", fill="both", expand=True)
    
    # Now add product widgets to the scrollable_frame instead of the products_tab
    conn = create_connection()
    cursor = conn.cursor()
    cursor.execute("SELECT ProductID, Name, Price FROM Product")
    products = cursor.fetchall()
    conn.close()
    
    # Add a label to display total amount
    total_amount_label = tk.Label(products_tab, text=f"Total Amount: {total_amount}")
    total_amount_label.pack()
    
    # Add a label to display cart items
    cart_items_label = tk.Label(products_tab, text="")
    cart_items_label.pack()
    
    for product in products:
        frame = tk.Frame(scrollable_frame)  # changed from products_tab to scrollable_frame
        tk.Label(frame, text=f"{product[1]} - Price: {product[2]}").pack(side='left')
        quantity_entry = tk.Entry(frame)
        quantity_entry.pack(side='left')
        add_button = tk.Button(frame, text="Add to Cart", command=lambda p=product[0], q=quantity_entry: add_to_cart(session_id, p, q.get(), total_amount_label, cart_items_label))
        add_button.pack(side='left')
        frame.pack()


global cart_items
cart_items = []

def add_to_cart(session_id, product_id, quantity, total_amount_label, cart_items_label):
    if not quantity.isdigit() or int(quantity) <= 0:
        messagebox.showerror("Error", "Invalid quantity")
        return

    try:
        conn = create_connection()
        cursor = conn.cursor()
        cursor.execute("INSERT INTO CartProduct (ShoppingSessionID, ProductID, Quantity) VALUES (%s, %s, %s)", (session_id, product_id, int(quantity)))
        conn.commit()

        # Get product name for display
        cursor.execute("SELECT Name FROM Product WHERE ProductID = %s", (product_id,))
        product_name = cursor.fetchone()[0]
        conn.close()

        # Update global cart items list
        cart_items.append(f"{product_name} x {quantity}")
        update_cart_items_display(cart_items_label)

        messagebox.showinfo("Success", "Product added to cart")
        update_total_amount_label(total_amount_label, session_id)
    except pymysql.Error as e:
        messagebox.showerror("Error", f"An error occurred: {e}")

def update_cart_items_display(cart_items_label):
    cart_items_label.config(text="\n".join(cart_items))
        
def products_tab(tab_control, session_id):
    products_tab = ttk.Frame(tab_control)
    tab_control.add(products_tab, text="Products")
    display_products(products_tab, session_id)

def display_order_summary(final_tab, order_id):
    conn = create_connection()
    cursor = conn.cursor()
    cursor.execute("SELECT p.Name, p.Price, cp.Quantity FROM OrderItem oi JOIN CartProduct cp ON oi.CartProductID = cp.CartProductID JOIN Product p ON cp.ProductID = p.ProductID WHERE oi.OrderID = %s", (order_id,))
    items = cursor.fetchall()
    conn.close()

    for item in items:
        tk.Label(final_tab, text=f"Product: {item[0]}, Quantity: {item[2]}, Total Price: {item[1] * item[2]}").pack()

def final_tab(tab_control, order_id):
    final_tab = ttk.Frame(tab_control)
    tab_control.add(final_tab, text="Final")
    display_order_summary(final_tab, order_id)

def start_shopping_session(customer_id):
    conn = create_connection()
    cursor = conn.cursor()
    cursor.execute("INSERT INTO ShoppingCart (CustomerID) VALUES (%s)", (customer_id,))
    conn.commit()
    session_id = cursor.lastrowid
    conn.close()
    return session_id

def create_order(customer_id, session_id):
    conn = create_connection()
    cursor = conn.cursor()
    
    cursor.execute("INSERT INTO `Order` (CustomerID, StatusID) VALUES (%s, 1)", (customer_id,))
    conn.commit()
    order_id = cursor.lastrowid

    cursor.execute("SELECT MAX(ShoppingSessionID) FROM ShoppingCart WHERE CustomerID = %s", (customer_id,))
    shopping_session_id = cursor.fetchone()[0]

    cursor.execute("INSERT INTO OrderItem (OrderID, ShoppingSessionID) VALUES (%s, %s)",
                   (order_id, session_id))
    
    conn.commit()
    conn.close()
    return order_id

def get_payment_types():
    conn = create_connection()
    cursor = conn.cursor()
    cursor.execute("SELECT TypeID, TypeName FROM PaymentType")
    payment_types = cursor.fetchall()
    conn.close()
    return payment_types


def create_shipment(customer_id):
    conn = create_connection()
    cursor = conn.cursor()

    cursor.execute("SELECT AddressID FROM Customer WHERE CustomerID = %s", (customer_id,))
    address_id = cursor.fetchone()[0]

    cursor.execute("INSERT INTO Shipment (AddressID, ShipmentDate, TrackingNumber) VALUES (%s, NULL, NULL)", (address_id,))
    conn.commit()

    shipment_id = cursor.lastrowid

    conn.close()
    return shipment_id

def open_main_app(customer):
    main_app = tk.Tk()
    main_app.title("Customer Portal")
    main_app.geometry("800x600") 
    
    # Adding a welcome message as a popup
    customer_name = f"{customer[1]} {customer[2]}"  # Assuming first and last name are in the 2nd and 3rd positions
    messagebox.showinfo("Welcome", f"Welcome, {customer_name}!")

    tab_control = ttk.Notebook(main_app)

    # Customer Tab
    customer_tab = ttk.Frame(tab_control)
    tab_control.add(customer_tab, text="Customer")
    # Description text for the Customer page
    description_text = """This is the Customer Page of the Online Shopping Customer Application. Here, you can view and update your personal information including your name, email, and phone number. Please use the 'Update' buttons next to each piece of information to make changes."""
    tk.Label(customer_tab, text=description_text, wraplength=700, justify='left',font=("Helvetica", 13)).pack(pady=10)
    display_customer_info(customer_tab, customer)

    # Address Tab
    address_tab = ttk.Frame(tab_control)
    tab_control.add(address_tab, text="Address")
    
    # Address Tab Description
    description_address_tab = """Welcome to the Address section of the Customer Portal. Here, you can view and update your address details. This includes your house number, street, city, state, and zipcode. To update any information, simply click the 'Update' button next to the field you wish to change, enter the new value, and confirm your update. Please note that keeping your address information up to date ensures accurate and prompt delivery of any items you purchase."""
    # Add this description to the top of the address tab
    tk.Label(address_tab, text=description_address_tab, wraplength=700, justify='left',font=("Helvetica", 13)).pack(pady=10)
    
    # Call display_address_info in the address tab
    display_address_info(address_tab, customer[0])
    
    # Products Tab
    products_tab = ttk.Frame(tab_control)
    tab_control.add(products_tab, text="Products")
    
    # Add the 'Start New Shopping Session' button at the top of the Products tab
    def initialize_shopping_session():
        global session_id
        session_id = start_shopping_session(customer[0])
        display_products(products_tab, session_id)

    tk.Button(products_tab, text="Start New Shopping Session", command=initialize_shopping_session).pack()

    # Create Order Tab
    create_order_tab = ttk.Frame(tab_control)
    tab_control.add(create_order_tab, text="Create Order")

    def create_new_order():
        order_id = create_order(customer[0], session_id)
        messagebox.showinfo("Success", f"Order {order_id} created successfully")

    tk.Button(create_order_tab, text="Create Order", command=create_new_order).pack()

    # Payment Tab
    payment_tab = ttk.Frame(tab_control)
    tab_control.add(payment_tab, text="Payment")

    payment_types = get_payment_types()
    payment_type_var = tk.StringVar()
    payment_type_var.set(payment_types[0][1])

    tk.Label(payment_tab, text="Payment Type:").pack()
    payment_type_menu = ttk.Combobox(payment_tab, textvariable=payment_type_var, values=[payment[1] for payment in payment_types])
    payment_type_menu.pack()


    tk.Label(payment_tab, text="Cardholder Name:").pack()
    cardholder_name_entry = tk.Entry(payment_tab)
    cardholder_name_entry.pack()

    tk.Label(payment_tab, text="Card Number:").pack()
    card_number_entry = tk.Entry(payment_tab)
    card_number_entry.pack()

    tk.Label(payment_tab, text="Security Number:").pack()
    security_number_entry = tk.Entry(payment_tab)
    security_number_entry.pack()

    tk.Label(payment_tab, text="Zipcode:").pack()
    zipcode_entry = tk.Entry(payment_tab)
    zipcode_entry.pack()

    def create_payment():
        selected_payment_type = payment_type_var.get()
        cardholder_name = cardholder_name_entry.get()
        card_number = card_number_entry.get()
        security_number = security_number_entry.get()
        zipcode = zipcode_entry.get()

        # Seçilen Payment Type'ın ID'sini alın
        selected_payment_type_id = None
        for payment in payment_types:
            if payment[1] == selected_payment_type:
                selected_payment_type_id = payment[0]
                break

        if selected_payment_type_id is None:
            messagebox.showerror("Error", "Invalid payment type selection")
            return

        conn = create_connection()
        cursor = conn.cursor()

        cursor.execute("INSERT INTO Payment (TypeID, CardholderName, CardNumber, SecurityNumber, Zipcode, PaymentStatusID) VALUES (%s, %s, %s, %s, %s, 2)",
                       (selected_payment_type_id, cardholder_name, card_number, security_number, zipcode))
        conn.commit()

        payment_id = cursor.lastrowid

        cursor.execute("UPDATE `Order` SET PaymentID = %s, OrderDate = NOW() WHERE CustomerID = %s AND OrderDate IS NULL", (payment_id, customer[0]))
        conn.commit()

        shipment_id = create_shipment(customer[0])

        cursor.execute("UPDATE `Order` SET PaymentID = %s, ShipmentID = %s, OrderDate = NOW() WHERE CustomerID = %s AND OrderDate IS NULL", (payment_id, shipment_id, customer[0]))
        conn.commit()

        cursor.execute("SELECT MAX(OrderID) FROM `Order` WHERE CustomerID = %s", (customer[0],))
        last_order_id = cursor.fetchone()[0]

        cursor.execute("UPDATE `Order` SET ShipmentID = %s WHERE OrderID = %s", (shipment_id, last_order_id))
        conn.commit()

        conn.close()

        messagebox.showinfo("Success", "Payment information saved successfully")
    
    tk.Button(payment_tab, text="Submit Payment", command=create_payment).pack()
    
    tab_control.pack(expand=1, fill="both")
    main_app.mainloop()
    
# Login Window
login_window = tk.Tk()
login_window.title("Customer Login")

tk.Label(login_window, text="Email:").pack()
email_entry = tk.Entry(login_window)
email_entry.pack()

tk.Label(login_window, text="Password:").pack()
password_entry = tk.Entry(login_window, show="*")
password_entry.pack()

login_button = tk.Button(login_window, text="Login", command=lambda: login(email_entry.get(), password_entry.get()))
login_button.pack()

login_window.mainloop()

/var/folders/sn/m91g82490jbfm36bn4bwq1m40000gn/T/ipykernel_2244/3859319026.py:77: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  img = img.resize((300, 300), Image.ANTIALIAS)
